# Human English Language Testing System 2

AI System that tests humans English language reading and writing skills. 

### In this version, 
Gradio is used to display the essay and questions.
Difficulty level can now be selected via a dropdown
Answer submission is now supported
Candidate get their grading and feedback from AI examiner

In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import json
import gradio as gr
from IPython.display import Markdown, display, update_display

In [ ]:
# constants

OPENROUTER_MODEL = "gpt-5-nano"

In [ ]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv("OPENROUTER_API_KEY")

gpt_base_url = "https://openrouter.ai/api/v1"

openrouter = OpenAI(api_key=api_key, base_url=gpt_base_url)

In [83]:
# System prompt

system = [{
        "role": "system",
        "content": """
    You are an English language examiner who tests humans on their reading skills. Difficulty level is how hard
    the essay comprehension is and how hard the questions are. It is from 1 to 4, 1 being the easiest and 4 being the hardest.
    The essay can be structured as title and body. Only paragraphs allowed, ho headers or subheaders.
    """,
    },]


In [23]:
def generate_essay(difficulty=1, topic=None, num_questions=3):
    return f"Generate an essay with difficulty level {difficulty} on {topic if topic else 'any topic'}. \
    Ask {num_questions} questions from the essay."

In [24]:
# Essay generation instructions


def get_question(difficulty=1, topic=None, num_questions=3):
    return f"""Generate an essay in with difficulty level {difficulty} on {topic if topic else 'any topic'}.
    Ask {num_questions} questions from the essay."""


def get_response_format():
    return """then return the essay and questions in this json format: 
      {
        "essay": "essay text in markdown format",
        "questions": ["question 1", "question 2", "question 3"]
      }
      """

In [ ]:
level = [("Easy", "1"), ("Intermediate", "2"), ("Advanced", "3"), ("Expert", "4")]
topics = [
    "AI",
    "Technology",
    "Science",
    "History",
    "Art",
    "Culture",
    "Sports",
    "Politics",
    "Economy",
    "Environment",
]

In [90]:
# business and ui logic

messages = system
current_question = 0
essay_text = ""
questions = []
answers = []


def submit_answers(final_answer):
    global current_question, answers
    if final_answer.strip():
        answers[current_question] = final_answer

    if current_question == len(questions) - 1:
        for i in range(len(questions)):
            messages.append({"role": "assistant", "content": questions[i]})
            messages.append({"role": "user", "content": answers[i]})

    messages.append(
        {
            "role": "user",
            "content": "Please grade the essay. 70+ A, 60-69 B, 50-59 C, 40-49 D, 0-39 F. After \
              show review as thus: the question, user answer, correct answer if wrong. format as \
                markdown in the form of a result sheet and Dont include extranous info",
        }
    )
    response = openrouter.chat.completions.create(
        model=OPENROUTER_MODEL, messages=messages
    )
    return response.choices[0].message.content


def next_question(answer):
    global current_question, answers
    has_next_question = current_question < len(questions) - 1 and answer.strip()

    if answer.strip():
        answers[current_question] = answer

    if has_next_question:
        current_question += 1

    return (
        essay_text,
        questions[current_question],
        "" if has_next_question else answers[-1],
    )


def start_examination(difficulty, topic, num_questions):
    global essay_text, questions, answers, current_question
    if essay_text:
        return essay_text, questions[current_question]

    messages.append(
        {
            "role": "user",
            "content": get_question(difficulty, topic, num_questions)
            + get_response_format(),
        }
    )

    response = openrouter.chat.completions.create(
        model=OPENROUTER_MODEL, messages=messages
    )
    response_text = response.choices[0].message.content
    response_json = json.loads(response_text)
    essay_text = response_json["essay"]
    questions = response_json["questions"]
    answers = [""] * len(questions)
    current_question = 0

    return essay_text, questions[current_question]


with gr.Blocks() as ui:
    with gr.Row():
        with gr.Column():
            difficulty = gr.Dropdown(choices=level, value="1", label="Difficulty")
        with gr.Column():
            topic = gr.Dropdown(choices=topics, value="AI", label="Topic")
    with gr.Row():
        with gr.Column():
            essay_text_container = gr.Markdown("*Essay will appear here...*")
        with gr.Column():
            question_text_container = gr.Markdown("*Question will appear here...*")
            answer_text = gr.Textbox(
                value="",
                container=None,
                interactive=True,
                placeholder="Type your answer here",
                lines=10,
            )
            with gr.Row():
                next_btn = gr.Button("Next Question")
                submit_btn = gr.Button("Submit")
            next_btn.click(
                next_question,
                inputs=[answer_text],
                outputs=[essay_text_container, question_text_container, answer_text],
            )
    with gr.Row():
        start_btn = gr.Button("Start Examination")
    with gr.Row():
        feedback_container = gr.Markdown("*Feedback will appear here...*")

    submit_btn.click(submit_answers, inputs=[answer_text], outputs=[feedback_container])

    start_btn.click(
        start_examination,
        inputs=[difficulty, topic, gr.State(value=4)],
        outputs=[essay_text_container, question_text_container],
    )

ui.launch()

* Running on local URL:  http://127.0.0.1:7902
* To create a public link, set `share=True` in `launch()`.
